In [60]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as XGBRegressor
from sklearn.model_selection import GridSearchCV
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import pickle
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor

In [61]:
df = pd.read_pickle('df_model')

In [62]:
df = df.dropna(subset =['final_years', 'final_make'])
df =df.drop(['make','make_title', 'year', 'years', 'title_lower', 'URL', 'bikeTitle', 'attributes',], axis =1)
df = df[df['final_years'].astype(str).str.isdigit()]
df['odometer'] = df.odometer.str.extract('(^\d*)')
df[["final_years"]] = df[['final_years']].apply(pd.to_numeric)
df['Price']= df['Price'].apply(lambda x: x.replace('$',''))
df['Price']= df['Price'].apply(lambda x: x.replace(',',''))
df[['Price']] = df[['Price']].apply(pd.to_numeric)
df[['odometer']] = df[['odometer']].apply(pd.to_numeric)
df =df.reset_index(drop=True)
df = df[df['Price']<50000]
df=df[df['Price']>600]
df = df[df['odometer']<100000]
df = df[df['final_years']>1960]
df = df[df['final_years']<2022]
df.drop("VIN", inplace =True, axis =1)
df =df.reset_index(drop=True)
df['condition'] = df['condition'].str.replace('excellentcryptocurrency ok', 'excellent')
df['condition'] = df['condition'].str.replace('like newcryptocurrency ok', 'like new')
df['condition'] = df['condition'].str.replace('goodcryptocurrency ok', 'good')
df['condition'] = df['condition'].str.replace('faircryptocurrency ok', 'fair')
df['condition'] = df['condition'].str.replace('newcryptocurrency ok', 'new')

In [63]:
df['fuel'] = df['fuel'].fillna(value='other')
df['paint color'] = df['paint color'].fillna(value='other')
df['title status'] = df['title status'].fillna(value='other')
df['transmission'] = df['transmission'].fillna(value='other')
df['condition'] = df['condition'].fillna(value='other')
df['engine displacement (CC)'] = df['engine displacement (CC)'].fillna(value='other')
df['type'] = df['type'].fillna(value='other')

In [64]:
df = df[df['engine displacement (CC)']!='other']

In [65]:
df['engine displacement (CC)']= df['engine displacement (CC)'].astype(float)
bins = [0,250,500,750,1000,1250,1500,1750,2000, np.inf]
names =['0-250', '251-500', '501-750', '751-1000', '1001- 1250', '1251-1500', '1501-1750',' 1751-2000', '2001+']

df['engine_displacement'] = pd.cut(df['engine displacement (CC)'], bins, labels=names)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5045 entries, 3 to 12483
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Price                     5045 non-null   int64   
 1   title                     5045 non-null   object  
 2   fuel                      5045 non-null   object  
 3   odometer                  5045 non-null   float64 
 4   paint color               5045 non-null   object  
 5   title status              5045 non-null   object  
 6   transmission              5045 non-null   object  
 7   condition                 5045 non-null   object  
 8   engine displacement (CC)  5045 non-null   float64 
 9   type                      5045 non-null   object  
 10  final_make                5045 non-null   object  
 11  final_years               5045 non-null   int64   
 12  engine_displacement       4999 non-null   category
dtypes: category(1), float64(2), int64(2), object(8)

In [67]:
df = df.drop('engine displacement (CC)', axis =1)

In [68]:
# Use train_test_split to create training data and testing data
X = df.drop(['Price', 'title'], axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [69]:
categoricals = ['fuel','paint color','title status', 'transmission', 'condition', 'type', 'final_make','engine_displacement']

In [70]:
pipeline1 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestRegressor(max_depth=30,
                                      n_estimators=50, n_jobs=-3,
                                        random_state=30)
)
results = cross_validate(pipeline1, X_train, y_train, return_train_score=True)
# pipeline1.fit(X_train, y_train)

# y_pred = pipeline1.predict(X_test)
# print('MAE:', mean_absolute_error(y_test, y_pred))
# print('R^2:', r2_score(y_test, y_pred))
# print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

In [71]:
print(results['test_score'].mean())

0.7366723287127709


In [72]:
pipeline1.fit(X,y)

Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['fuel', 'paint color', 'title status',
                                     'transmission', 'condition', 'type',
                                     'final_make', 'engine_displacement'])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=30, n_estimators=50, n_jobs=-3,
                                       random_state=30))])

In [14]:
with open("model_1.pkl", 'wb') as f:
    pickle.dump(pipeline1, f)

In [74]:
param_grid = {
    'randomforestregressor__max_depth': [2,5,6,10,30,50],
    'randomforestregressor__n_jobs': [10,-5,-3,-1, 0,1, 3,5,10],
    'randomforestregressor__n_estimators': [10, 50, 100, 500 ],
}

In [75]:
rfr =RandomForestRegressor()

In [76]:
pipeline5 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    rfr)

In [80]:
grid_rfr = GridSearchCV(pipeline5, param_grid, cv=5, n_jobs=1, return_train_score=True)

In [81]:
results_rfr = grid_rfr.fit(X_train, y_train)

/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 966, in __call__
    n_jobs = self._initialize_backend()
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel

/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 966, in __call__
    n_jobs = self._initialize_backend()
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel

/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 966, in __call__
    n_jobs = self._initialize_backend()
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel

/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 966, in __call__
    n_jobs = self._initialize_backend()
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel

/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 966, in __call__
    n_jobs = self._initialize_backend()
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel

...

/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda

/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worke

/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 966, in __call__
    n_jobs = self._initialize_backend()
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel

/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 966, in __call__
    n_jobs = self._initialize_backend()
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel

/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 966, in __call__
    n_jobs = self._initialize_backend()
  File "/Users/avijames/anaconda3/lib/python3.8/site-packages/joblib/parallel

In [82]:
results_rfr.best_params_

{'randomforestregressor__max_depth': 30,
 'randomforestregressor__n_estimators': 500,
 'randomforestregressor__n_jobs': -3}

## Using the best parameters to make the finished pipeline

In [89]:
pipeline = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestRegressor(max_depth=30,
                                      n_estimators=500, n_jobs=-3,
                                        random_state=30))

In [90]:
pipeline6.fit(X_train, y_train)

y_pred = pipeline6.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R^2:', r2_score(y_test, y_pred))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

MAE: 1524.4666542766442
R^2: 0.7489547277036487
RMSE: 2317.9103193155943


### Pickling the pipeline to use for our deployment app 

In [88]:
with open("model_2.pkl", 'wb') as f:
    pickle.dump(pipeline6, f)

In [85]:
results = cross_validate(pipeline6, X_train, y_train, return_train_score=True)

In [86]:
print(results['test_score'].mean())

0.7432976432175604


In [15]:
pipeline2 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    XGBRegressor())

# pipeline2.fit(X_train, y_train)
# y_pred = pipeline2.predict(X_test)
# print('MAE:', mean_absolute_error(y_test, y_pred))
# print('R^2:', r2_score(y_test, y_pred))
# print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

In [16]:
pipeline4 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    XGBRegressor(learning_rate=0.1,
                max_depth=6,
                min_child_weight=2,
                n_estimators =100,
                subsample=0.9))
results = cross_validate(pipeline1, X_train, y_train, return_train_score=True)

# pipeline4.fit(X_train, y_train)
# y_pred = pipeline4.predict(X_test)
# print('MAE:', mean_absolute_error(y_test, y_pred))
# print('R^2:', r2_score(y_test, y_pred))
# print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

In [17]:
print(results['test_score'].mean())

0.7366723287127709


In [43]:
param_grid = {
    'xgbregressor__learning_rate': [0.1, 0.5, 1],
    'xgbregressor__max_depth': [2,5,6,10],
    'xgbregressor__min_child_weight': [1, 2, 3],
    'xgbregressor__subsample': [0.5, 0.7, .9],
    'xgbregressor__n_estimators': [10, 50, 100, 500 ],
}

In [44]:
xgb =XGBRegressor()

In [45]:
pipeline3 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    xgb)

In [46]:
grid_clf = GridSearchCV(pipeline3, param_grid, cv=5, n_jobs=1, return_train_score=True)

In [47]:
results = grid_clf.fit(X_train, y_train)
# y_pred = pipeline3.predict(X_test)
# print('MAE:', mean_absolute_error(y_test, y_pred))
# print('R^2:', r2_score(y_test, y_pred))
# print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
pipeline4.fit(X,y)

In [ ]:
with open("model.pkl", 'wb') as f:
    pickle.dump(pipeline4, f)

In [48]:
results.best_params_

{'xgbregressor__learning_rate': 0.1,
 'xgbregressor__max_depth': 6,
 'xgbregressor__min_child_weight': 2,
 'xgbregressor__n_estimators': 100,
 'xgbregressor__subsample': 0.9}